In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import re,os
import glob, traceback
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

## 0. Read all csv files

In [2]:
df = pd.read_csv('/data2/julina/scripts/tweets/cleaned_data_by_year/2020.csv')
df.drop_duplicates(subset=['text', 'created_at'], inplace=True)
df = df.loc[:, ~df.columns.str.match('Unnamed')]

df_r = pd.read_csv('/data2/julina/scripts/tweets/cleaned_data_by_year/2020_race.csv')
df_r.drop_duplicates(subset=['text', 'created_at'], inplace=True)
df_r = df_r.loc[:, ~df_r.columns.str.match('Unnamed')]

df_2020 = pd.merge(df, df_r[['id', 'user_id', 'race']],  how='left', on=['id','user_id'])
df_2020.shape

df.shape, df_r.shape, df_2020.shape

((3502171, 17), (2275943, 18), (3502171, 18))

In [3]:
drug_keywords_map= {
# 'tobacco' : ['nicotine', 'tobacco', 'cigarette', 'cigarrette', 'cigar', 'bidis', 'snuff'],
'alcohol' : ['liquor', 'beer', 'wine'],
# 'cannabinoids' : ['marijuana', 'blunt', 'dope', 'ganja', 'grass', 'herb', 'joint', 'bud', 'mary jane', 'pot', 'reefer', 'green', 'trees', 'smoke', 'sinsemilla', 'skunk', 'weed','hashish', 'boom', 'gangster', 'hash', 'hash oil', 'hemp'],
# 'opioids' : ['heroin', 'smack', 'horse', 'brown sugar', 'dope', 
#              # 'H',
#              'junk', 'skag', 'skunk', 'white horse', 'China white','opium', 'laudanum', 'paregoric', 'big O', 'black stuff', 'block', 'gum', 'hop'],
# 'stimulants': ['cocaine', 'hydrochloride', 'blow', 'bump',
#                # 'C',
#                 'candy', 'Charlie', 'coke', 'crack', 'flake', 'rock', 'snow', 'toot','amphetamine', 'Biphetamine', 'Dexedrine', 'bennies', 'black beauties', 'crosses', 'hearts', 'LA turnaround', 'speed', 'truck drivers', 'uppers','methamphetamine', 'Desoxyn','meth', 'ice', 'crank', 'chalk', 'crystal', 'fire', 'glass', 'go fast', 'speed'],
# 'club_drugs':['mdma', 'ecstasy', 'adam', 'clarity', 'eve', "lover's speed", 'peace', 'uppers','flunitrazepam', 'rohypnol', 'forget-me pill', 'mexican valium', 'r2', 'roach', 'roche', 'roffles', 'roofinol', 'rope', 'rophies','ghb', 'Gamma-hydroxybutyrate', 
#               # 'G',
#               'Georgia home boy', 'grievous bodily harm', 'liquid ecstasy', 'soap', 'scoop', 'goop', 'liquid X'],
# 'dissociative_drugs':['ketamine', 'Ketalar SV', 'cat Valium',
#                       # 'K',
#                       'Special K', 'vitamin K','pcp and analogs', 'phencyclidine' 'angel dust', 'boat', 'hog', 'love boat', 'peace pill','salvia divinorum', 'salvia', 'shepherdess’s herb', 'maria pastora', 'magic mint', 'sally-d','dextromethorphan', 'dxm',  'robotripping', 'robo', 'triple'],
# 'hallucinogens':['lsd','Lysergic acid diethylamide', 'acid', 'blotter', 'cubes', 'microdot', 'yellow sunshine', 'blue heaven', 'mescaline', 'Buttons', 'cactus', 'mesc', 'peyote', 'psilocybin', 'Magic mushrooms', 'purple passion', 'shrooms', 'little smoke'],
# 'other_compounds':['anabolic_steroids', 'Anadrol', 'Oxandrin', 'Durabolin', 'Depo-Testosterone', 'Equipoise', 'roids', 'juice', 'gym candy', 'pumpers','inhalants', 'Solvents', 'paint thinners', 'gasoline', 'glues', 'gasses', 'butane', 'propane', 'aerosol propellants', 'nitrous oxide',  'nitrites' ,'isoamyl', 'isobutyl', 'cyclohexyl','laughing gas', 'poppers', 'snappers', 'whippets'],
# 'prescription_medications':['cns_depressants', 'stimulants', 'opioid pain relievers', 'OxyContin','Oxycodone', 'Vicodin', 'Norco', 'Lortab', 'Hydrocodone', 'Acetaminophen', 'Percocet ', 'Oxycodone', 'Acetaminophen','Tramadol','Codeine','Morphine','Methadone','Demerol', 'meperidine','Acetaminophen','Tylenol', 'Excedrin', 'Vanquish','Aspirin', 'Bayer', 'Bufferin', 'Ecotrin', 'Excedrin', 'Vanquish','Diclofenac', 'Voltaren Gel','Ibuprofen', 'Advil', 'Motrin IB','Naproxen', 'Aleve']
}

drug_cols = drug_keywords_map.keys()

In [4]:
# https://nida.nih.gov/sites/default/files/cadchart.pdf
for keyword, words in drug_keywords_map.items():
    pattern = fr'\b(?:{"|".join(words)})\b'
    df_2020[keyword] = df_2020['text'].str.contains(pattern, case=False).astype(int)

In [5]:
df_2020['drug_type'] = df_2020[drug_cols].apply(lambda row: next((col for col, val in zip(drug_cols, row) if val == 1), None), axis=1)

## Theme analysis

In [6]:
covid = ['covid', 'corona', 'coronavirus', 'outbreak', 'spread', 'viral','virus', 'flu', 'infect','pcr test', 'national emergency', 'wuhan', 'CDC', 'ncov', 'SARS', 'mask', 'sanitizer',
         'toll', 'new disease', 'news', 'positive', 'test', 'cases',  'dangerous year', 'fake year', 'fda', 'flatten', 'cov2', 'quarantine', 'lockdown',
         'pandemic', 'epidemic', 'ventilator', 'symptomatic', 'incubation', 'transmission', 'immune', 'vaccine',
          'hard to breath', 'shortness of breath','start nose', 'variant']

economic =  ['job', 'job loss', 'jobless', 'no job', 'lost job', 'income', 'salary', 'paycheck', 'money', 'bills', 'bill raise',  'check', 'saving', 'wage','fulltime',
             'unemployed', 'economy', 'layoff', 'recession', 'stimulus', 'evict', 'laid', 'bankrupt', 'owe', 'afford',  'eviction', 'credit', 'debt', 'loan', 'financial crisis', 'inflation',
              'poor', 'poverty', 'normalize risk', 'cost', 'homeless', 'rent', 'mortgage', 'no food',  'no shelter', 'wallet', 'hungry street', 'fda']

social = ['lockdown', 'bully', "loneliness", "emptiness", "lonely", "distancing", "insecurities", "no one care", "trap", "feel ignore", "ignored", 'bored', 
         'quarantine', 'alone', 'stay home', 'stayhome', 'stay safe', 'stayathome', 'single', 'solo', 'lockup', 'disconnect', 
          'silence', 'off limit',  'frayed', 'indoor']

mentalhealth = ['sad', 'angry', 'mad', 'tension', 'depress', 'tire', 'exhausted', 'fear', 'scared', 'stress', 'worry', 'disappear', 'selfharm', 'crisis', 'dizzi', 'no hunger', 'no sleep','no hope', 'hopeless', 'loneliness', 'no will', 'quit life', 'no motivation', 'no motif', 'no focus', 'no dedication', 'cannot focus',
                'panic', 'panic attack', 'rapid heartbeat','heartbeat fast', 'suicidal thought', 'ending my life', 'no reason to live', 'feeling worthless',  'seeking a way out', 'feel like a burden', 'struggling to cope', 'lack of purpose', 'wanting the pain to stop',
                'mental pain', 'mental anguish', 'feel pressure', 'feel trapped', 'desperation', 'broken inside', 'losing the will to live','dark thought', 'depression', 'anxiety', 'ptsd', 'bipolar',
               'intrusive', 'trauma', 'burnout', 'grief', 'dyingfeel', 'sorrow']

supply_disruption = ['steal', 'deliver', 'refuse', 'small rate', 'sellout', 'sell bath', 'sell illegal', 'sell weed', 'cheap deal', 'box sale', 'sale', 'order', 'order pack', 'cut safe', 'strong', 'weak', 'legal high', 
                     'dangerous', 'toxic', 'industry', 'contaminated', 'local', 'import', 'international', 'darknet',  'market', 'stock alert', 'batch', 'customs', 'seizure', 'permit', 'lack', 'shortage', 'crisis', 'sale', 'buy drug', 'supply']

medical_disruption = ['health crisis', 'shut down', 'cut off', 'on hold',  'locked out', 'dead end', 
                      'dry', 'mia',  
                      'rehab', 'virtual', 'telehealth', 'teletherapy', 'telemedicine', 'appointment',  'close clinic', 'limited hours',
                       'waitlist', 'treatment full', 'medical line', 'freeze', 'overload', 'emergency','drought', 'no help line', 'pharmacy', 'customer service']
factor_keywords_map = {
                 'covid': covid,
                 'economic': economic,
                 'social' : social,
                 'mentalhealth': mentalhealth,
                 'supply_disruption': supply_disruption,
                 'medical_disruption': medical_disruption
                }

for keyword, words in factor_keywords_map.items():
    pattern = fr'\b(?:{"|".join(words)})\b'
    df_2020[keyword] = df_2020['text'].str.contains(pattern, case=False).astype(int)

In [7]:
df_2020['created_at'] = pd.to_datetime(df_2020['created_at'], format='%a %b %d %H:%M:%S %z %Y')

In [9]:
sub_data = df_2020[df_2020['drug_type']=='alcohol']

In [10]:
pandemic= sub_data[(sub_data['created_at'] >= '2020-03-15') & (sub_data['created_at'] <= '2020-03-31')]
pandemic.shape

(26550, 25)

In [88]:
results = pd.DataFrame()
periods = [
    ('2020-03-01', '2020-03-14'),
    ('2020-03-15', '2020-03-31'),
    ('2020-04-01', '2020-04-14'),
    ('2020-04-15', '2020-04-30'),
    ('2020-05-01', '2020-05-14'),
    ('2020-05-15', '2020-05-31')
]
for i, (start_date, end_date) in enumerate(periods, 1):
    period_data = sub_data[(sub_data['created_at'] >= start_date) & (sub_data['created_at'] <= end_date)]
    counts = period_data[['covid', 'economic', 'social', 'mentalhealth', 'supply_disruption', 'medical_disruption']].sum()
    percentages = (counts / len(period_data)) * 100
    combined = counts.astype(str) + ' (' + percentages.round(2).astype(str) + '%)'
    combined.name = f'wk {start_date}'
    
    if results.empty:
        results = pd.DataFrame(combined)
    else:
        results = results.join(pd.DataFrame(combined))

results.T

,covid,economic,social,mentalhealth,supply_disruption,medical_disruption
wk 2020-03-01,1434 (11.15%),606 (4.71%),245 (1.9%),157 (1.22%),374 (2.91%),41 (0.32%)
wk 2020-03-15,3213 (12.1%),427 (1.61%),2275 (8.57%),1226 (4.62%),1966 (7.4%),1091 (4.11%)
wk 2020-04-01,1105 (5.32%),323 (1.55%),706 (3.4%),219 (1.05%),789 (3.8%),131 (0.63%)
wk 2020-04-15,1022 (4.86%),389 (1.85%),871 (4.14%),198 (0.94%),787 (3.74%),97 (0.46%)
wk 2020-05-01,2151 (10.46%),593 (2.88%),795 (3.86%),202 (0.98%),1145 (5.57%),106 (0.52%)
wk 2020-05-15,505 (2.79%),293 (1.62%),286 (1.58%),132 (0.73%),654 (3.61%),82 (0.45%)


## Topic analysis below

In [ ]:
df = pandemic[pandemic['mentalhealth']==1]
# df

In [90]:
pandemic.groupby('gender').size()

gender
female    12169
male      14381
dtype: int64

In [77]:
df.groupby('sentiment').size(), pandemic.groupby('sentiment').size()

(sentiment
 neg    1155
 neu      20
 pos      51
 dtype: int64,
 sentiment
 neg     5757
 neu    10117
 pos    10676
 dtype: int64)

In [38]:
df.groupby('age').size(), df.groupby('gender').size()

(age
 19-29    395
 30-39    244
 <=18     381
 >=40     206
 dtype: int64,
 gender
 female    627
 male      599
 dtype: int64)

## LDA

In [65]:
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already downloaded
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jmaharja/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [66]:
words_to_remove = set(['httpurl', 'hashtag', 'user'])
stop_words = set(stopwords.words('english')).union(words_to_remove)

In [67]:
df['cleaned_text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

In [68]:
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), max_df=0.75, min_df=0.001, use_idf=True)
posts_tfidf_bowdff = vectorizer.fit_transform(df['cleaned_text'].values.astype('U'))

In [75]:
lda_model=LatentDirichletAllocation(n_components=4,learning_method='online',random_state=42,max_iter=1) 
lda_top = lda_model.fit_transform(posts_tfidf_bowdff)

In [89]:
# vocab = vectorizer.get_feature_names()
vocab = vectorizer.get_feature_names_out()
for i, comp in enumerate(lda_model.components_):
     vocab_comp = zip(vocab, comp)
     sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:20]
     print("Topic "+str(i)+": ")
     for t in sorted_words:
        print(t[0], end=" ")
     print('\n')

Topic 0: 
wine coronavirus coronavirus crisis beer anxiety denver https bar limit staff month beer mat pay nearly 100000 wage coronavirus crisis https spend nearly mat pay bar staff martin spend wage 

Topic 1: 
beer wine sad buy drink worry mad fear panic get bottle store close panic buy close really run liquor store close drink beer still little 

Topic 2: 
wine beer panic drink go mad people bottle bottle wine get drown one stress glass panic buying buying life okay people found panic found bottle 

Topic 3: 
1920 alcohol essential business national liquor store essential store essential 1920 alcohol prohibit 2020 national health prohibit 2020 liquor alcohol prohibit 2020 alcohol prohibit health crisis 1920 2020 liquor 2020 liquor store business national health national store essential business prohibit 2020 business national national health crisis 



## BERT Topic

In [20]:
from bertopic import BERTopic

/home/jmaharja/anaconda3/envs/gpu2/lib/python3.10/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/jmaharja/anaconda3/envs/gpu2/lib/python3.10/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/jmaharja/anaconda3/envs/gpu2/lib/python3.10/site-packages/umap/d

In [23]:
docs = df.cleaned_text.tolist()

In [24]:
from sentence_transformers import SentenceTransformer
# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(docs, show_progress_bar=True)

Batches: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 830/830 [00:18<00:00, 45.62it/s]


In [ ]:
from umap import UMAP
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine', random_state=42)

In [ ]:
from hdbscan import HDBSCAN
hdbscan_model = HDBSCAN(min_cluster_size=150, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# stop_words = stop_words + ['httpurl', 'hashtag', 'user']
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 2))

In [ ]:
# import openai
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, OpenAI, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
# pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# GPT-3.5
# openai.api_key = "sk-..."
# prompt = """
# I have a topic that contains the following documents: 
# [DOCUMENTS]
# The topic is described by the following keywords: [KEYWORDS]

# Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
# topic: <topic label>
# """
# openai_model = OpenAI(model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    # "OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model,
    # "POS": pos_model
}


In [ ]:
from bertopic import BERTopic

topic_model = BERTopic(
  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=20,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(docs, embeddings)

# Show topics
# topic_model.get_topic_info()

In [ ]:

# # GPT-3.5
# client = openai.OpenAI(api_key="sk-proj-V9eUaxTcHBwbuk09RvpHT3BlbkFJiONTDMtF1Nb9SWIfmWIn")
# prompt = """
# I have a topic that contains the following documents: [DOCUMENTS]
# The topic is described by the following keywords: [KEYWORDS]

# Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
# topic: <topic label>
# """
# openai_model = OpenAI(client,  prompt=prompt)

# topic_model = BERTopic(representation_model=openai_model)

In [ ]:
freq = topic_model.get_topic_info()
freq.shape

In [ ]:
freq

In [ ]:
# topic_model.get_topic(-1)

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

In [ ]:
freq.iloc[-1]['Representative_Docs']

In [ ]:
df[df['topic']==-1]

In [ ]:
topic_model.reduce_topics(docs, nr_topics=2)